In [35]:
import pandas as pd
import numpy as np
from sqlalchemy import MetaData, create_engine, Table, Column, ForeignKey
from sqlalchemy import Integer, String, Date, Float, Time
from sqlalchemy import exc

In [12]:
#Constantes
DB = "tp312"
HOST = "localhost"
PORT = "5432"
PASS = "password"
USER = "postgres"

![Diagrama]("Inmobiliaria.png"

In [13]:
url_conect = USER + ':' + PASS + '@' + HOST+ ':' + PORT + '/'+ DB
engine = create_engine('postgresql://' + url_conect)

In [27]:
metadata = MetaData()


estate = Table('estate', metadata,
              Column('id', Integer, primary_key=True),
              Column('category', String),
              Column('area', Float, nullable=False),
              Column('city', String, nullable=False),
              Column('province', String, nullable=False),
              Column('rooms', Integer, nullable=False),
              Column('bedrooms', Integer, nullable=False),
              Column('garage', Integer),
              Column('meters', Float, nullable=False))

customer = Table('customer', metadata,
                Column('id', Integer, primary_key=True),
                Column('name', String, nullable=False),
                Column('surname', String, nullable=False),
                Column('budget', Float),
                Column('address', String),
                Column('city', String),
                Column('phone', String))

visit = Table('visit', metadata, 
             Column('id', Integer, primary_key=True),
             Column('duration', Integer, nullable=False))

sale = Table('sale', metadata, 
            Column('id', Integer, primary_key=True),
            Column('agreed_price', Float, nullable=False),
            Column('status', String, nullable=False))

fact = Table('fact', metadata,
            Column('id', Integer, primary_key=True),
            Column('id_estate', Integer, ForeignKey('estate.id')),
            Column('id_cust', Integer, ForeignKey('customer.id')),
            Column('date', Date, nullable=False),
            Column('id_visit', Integer, ForeignKey('visit.id'), nullable=True),
            Column('id_sale', Integer, ForeignKey('sale.id'), nullable=True))


In [28]:
metadata.create_all(engine)

Carga de Datos
--------------

In [36]:
try:
    df_estate = pd.read_csv('dataTp3/Inmobilaria/Estate.csv')
    df_estate.to_sql('estate', engine, index=None, if_exists="append")
    
except exc.IntegrityError:
    print("Los datos de Estate ya han sido cargados en la BD")

Los datos ya han sido cargados en la BD


In [69]:
try:
    df_customer = pd.read_csv('dataTp3/Inmobilaria/Customer.csv')
    df_customer.to_sql('customer', engine, index=None, if_exists="append")
except exc.IntegrityError:
    print("Los datos de Customer ya han sido cargados en la BD")  

Los datos de Customer ya han sido cargados en la BD


In [68]:
try:
    df_visit = pd.read_csv('dataTp3/Inmobilaria/Visit.csv')
    df_visit.to_sql('visit', engine, index=None, if_exists="append")
except exc.IntegrityError:
    print("Los datos de Visit ya han sido cargados en la BD")

Los datos de Visit ya han sido cargados en la BD


In [67]:
try:
    df_sale = pd.read_csv('dataTp3/Inmobilaria/Sale.csv')
    df_sale.to_sql('sale', engine, index=None, if_exists="append")
except exc.IntegrityError:
    print("Los datos de Sale ya han sido cargados en la BD")

Los datos de Sale ya han sido cargados en la BD


In [40]:
try:
    df_fact = pd.read_csv('dataTp3/Inmobilaria/Fact.csv')
except exc.IntegrityError:
    print("Los datos de Fact ya han sido cargados en la BD")

,id,id_estate,id_cust,date,id_visit,id_sale
0,1,1,1,2006-12-17,NaN,NaN
1,2,2,2,2008-05-09,NaN,NaN
2,3,3,3,2001-03-14,NaN,NaN
3,4,4,4,2009-11-08,NaN,NaN
4,5,5,5,2015-06-09,NaN,NaN
5,6,6,6,2010-01-07,NaN,NaN
6,7,7,7,2015-05-22,NaN,NaN
7,8,8,8,2013-08-19,NaN,NaN
8,9,9,9,2005-06-14,NaN,NaN
9,10,10,10,2016-06-09,NaN,NaN


,id_visit
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN
9,NaN
